In [2]:
pip install google-generativeai jieba_zh_TW pandas opencc-python-reimplemented cnsenti xgboost langchain-community openai 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scipy==1.7.3 gensim==3.8.1

     ---------------------------------------- 0.0/23.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/23.4 MB ? eta -:--:--
     --------------------------------------- 0.0/23.4 MB 217.9 kB/s eta 0:01:48
     --------------------------------------- 0.0/23.4 MB 217.9 kB/s eta 0:01:48
     --------------------------------------- 0.0/23.4 MB 163.4 kB/s eta 0:02:23
     --------------------------------------- 0.1/23.4 MB 286.7 kB/s eta 0:01:22
     --------------------------------------- 0.1/23.4 MB 344.8 kB/s eta 0:01:08
     --------------------------------------- 0.1/23.4 MB 327.4 kB/s eta 0:01:12
     --------------------------------------- 0.2/23.4 MB 419.0 kB/s eta 0:00:56
     --------------------------------------- 0.2/23.4 MB 461.0 kB/s eta 0:00:51
     --------------------------------------- 0.2/23.4 MB 464.5 kB/s eta 0:00:50
      -------------------------------------- 0.3/23.4 MB 561.6 k

In [1]:
pip install numpy==1.22.4

  Using cached numpy-1.22.4-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
Using cached numpy-1.22.4-cp310-cp310-win_amd64.whl (14.7 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.9.1 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.


In [1]:
import jieba_zh_TW
import jieba_zh_TW.posseg as pseg
import re
import pandas as pd
import opencc
from cnsenti import Emotion
import pickle
from gensim.models.word2vec import Word2Vec
import numpy as np
import xgboost as xgb
from decimal import Decimal, ROUND_HALF_UP
import warnings
warnings.filterwarnings("ignore")

def sent_seg(paragraph):
    sents = re.findall(u'[^！？。\.\！\？\…]+[！？。\.\！\？\…]?', paragraph, flags=re.U)
    return sents

def BS_prediction(intro):
    data_path = "C:/Users/hhfeng/Desktop/智慧文案輔助系統/BS Level Prediction/"
    cc_ts = opencc.OpenCC('tw2s')
    emotion = Emotion()
    
    df = pd.read_csv(data_path+"Keywords_Features.csv", encoding="utf-8")
    kws = list(df["Keywords_Features"])
    
    ws_results = jieba_zh_TW.cut(intro)
    clean_ws_results = []
    single_alph = list('abcdefghijklmnopqrstuvwxyz')
    single_u_alph = list('abcdefghijklmnopqrstuvwxyz'.upper())
    al_single_alph = list('ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚ')
    al_single_u_alph = list('ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ')
    al_punc = list('~～!！（）()｛｝：:＂"＜＞<>？?’［］[]【】「」『』〔〕，,。.、·‧﹒．﹐﹑；:;﹔＋+＝=＆&《 》〈 〉◦•﹞﹝﹡{}')
    years = [str(y) for y in range(1700, 2101)]
    m_years = [str(y) for y in range(50, 131)]

    model = pickle.load(open(data_path+"(jieba_notin)xgb_067prob_kw2v_ttr_pos_mls_senti.pickle.dat", "rb"))
    w2v_model = Word2Vec.load(data_path+"(jieba_notin)kingstone_word2vec_d300_unibitri_mincount2.model")


    for w in ws_results:
        if w == "hellip" or w == "mdash" or w == "\n" or w == "hellip;" or w == "mdash;" or w == "bull" or w == "&" or w == "&&" or w == "&&&" or w == "rarr" or w == "／" or w == "middot" or w == "】" or w == "【" or w == "middot" or w == "․" or w == "∕" or w == "color" or w == "rgb" or w == "Arial" or w == "height" or w == "size" or w == "family" or w == "明體" or w == "新細" or w == 'sans' or w == 'Helvetica' or w == 'Verdana' or w == 'serif' or w == '17px' or w == 'small' or w == 'border' or w == 'sizing' or w == ';' or w == "：" or w == "\u3000" or w == "．"or w == '\uf050' or w == '&&&&' or w == '&&&&&' or w == '&&&&&&' or w == '&&&&&&&' or w == '&&&&&&&&' or w == '&#' or w == 'ｍ' or w == 'ｐ' or w == '&&&&&&&&&' or w == '\x84' or w == 'pstyle' or w == 'margin' or w == 'divstyle' or w == 'fontsize' or w == 'align' or w == 'yellownonerepeatscroll0':
            pass
        elif w in single_alph or w in al_single_alph or w in single_u_alph or w in al_single_u_alph or w in years or w in al_punc or w in m_years:
            pass
        elif re.search("uf", str(w.encode('unicode_escape'))) or re.search('px', w) or re.search('&', w) or re.search("page", w):
            pass
        elif w == '':
            pass
        else:
            clean_ws_results.append(w)
    
    intro_segs_uni = []
    intro_segs_bi = []
    intro_segs_tri = []
    
    intro_segs_uni.append("/".join(clean_ws_results))
    
    tuple_bi_ = list(zip(clean_ws_results, clean_ws_results[1:]))
    bi_ = []
    for t in tuple_bi_:
        b = t[0]+"_"+t[1]
        bi_.append(b)
    intro_segs_bi.append("/".join(bi_))
    
    tuple_tri_ = list(zip(clean_ws_results, clean_ws_results[1:], clean_ws_results[2:]))
    tri_ = []
    for t in tuple_tri_:
        tt = t[0]+"_"+t[1]+"_"+t[2]
        tri_.append(tt)
    intro_segs_tri.append("/".join(tri_))
    
    kw_result_df = pd.DataFrame()
    uni_words = "".join(intro_segs_uni).split("/")
    bi_words = "".join(intro_segs_bi).split("/")
    tri_words = "".join(intro_segs_tri).split("/")
    
    mls = round(len(uni_words)/len(sent_seg(intro)), 3)
    T = len(list(set(uni_words)))
    ttr = round(len(list(set(uni_words)))/len(intro_segs_uni), 3)
    
    kw_result_df["Mean_Length_of_Sentence"] = [float(mls)]
    kw_result_df["Type"] = [float(T)]
    kw_result_df["TTR"] = [float(ttr)]
    
    intro_pos = []
    
    word_pos = pseg.cut("".join("".join(intro_segs_uni).split("/")))
    for word, pos in word_pos:
        intro_pos.append(pos)
    
    universal_ps = []
    content_words = []
    verb_ws = []
    noun_ws = []
    adj_ws = []
    adv_ws = []
    conj_ws = []
    pron_ws = []
    adp_ws = []
    prt_ws = []
    num_ws = []
    
    for pst in intro_pos:
        if pst == 'd' or pst == "dg" or pst == "h" or pst == "t" or pst == "zg" or pst == 'z':
            universal_ps.append("ADV")
            content_words.append("ADV")
            adv_ws.append("ADV")
        elif pst == 'a' or pst == 'ad' or pst == "b" or pst == "an" or pst == "ag" or pst == "i" or pst == 'l':
            universal_ps.append("ADJ")
            content_words.append("ADJ")
            adj_ws.append("ADJ")
        elif pst == 'j' or pst == 'mg' or pst == 'n' or pst == 'ng' or pst == 'nr' or pst == 'nrfg' or pst == 'nrt' or pst == 'ns' or pst == 'nt' or pst == 'nz' or pst == 's' or pst == 'tg' or pst == 't':
            universal_ps.append("NOUN")
            content_words.append("NOUN")
            noun_ws.append("NOUN")
        elif pst == "df" or pst == 'f' or pst == 'v' or pst == 'vg' or pst == 'vd' or pst == 'vi' or pst == 'vn' or pst == 'vq':
            universal_ps.append("VERB")
            content_words.append("VERB")
            verb_ws.append("VERB")
        elif pst == 'c':
            universal_ps.append("CONJ")
            conj_ws.append("CONJ")
        elif pst == 'k' or pst == 'mq' or pst == 'r' or pst == 'rg' or pst == 'rr' or pst == 'rz':
            universal_ps.append("PRON")
            pron_ws.append("PRON")
        elif pst == 'm' or pst == 'q':
            universal_ps.append("NUM")
            num_ws.append("NUM")
        elif pst == 'u' or pst == 'ud' or pst == 'uj' or pst == 'uv' or pst == 'ug' or pst == 'ul' or pst == 'uz' or pst == 'y':
            universal_ps.append("PRT")
            prt_ws.append("PRT")
        elif pst == 'p':
            universal_ps.append("ADP")
            adp_ws.append("ADP")
    
    pos_types = ["ADV", "ADJ", "NOUN", "VERB", "CONJ", "PRON", "NUM", "PRT", "ADP"]
    for p in pos_types:
        kw_result_df[p+"_Ratio"] = [float(round(universal_ps.count(p)/len(universal_ps), 3))]
    
    kw_result_df["Lexical_Density"] = [round(len(content_words)/len(universal_ps), 3)]
    if len(content_words) == 0:
        kw_result_df["Lexical_Variation"] = [0]
        kw_result_df["Noun_Variation"] = [0]
        kw_result_df["Adjective_Variation"] = [0]
        kw_result_df["Adverb_Variation"] = [0]
        kw_result_df["Modifier_Variation"] = [0]
    else: 
        kw_result_df["Lexical_Variation"] = [round(len(list(set(content_words)))/len(content_words), 3)]
        kw_result_df["Noun_Variation"] = [round(len(list(set(noun_ws)))/len(content_words), 3)]
        kw_result_df["Adjective_Variation"] = [round(len(list(set(adj_ws)))/len(content_words), 3)]
        kw_result_df["Adverb_Variation"] = [round(len(list(set(adv_ws)))/len(content_words), 3)]
        kw_result_df["Modifier_Variation"] = [round((len(list(set(adv_ws)))+len(list(set(adj_ws))))/len(content_words), 3)]
    
    if len(verb_ws) == 0: 
        kw_result_df["Verb_VariationI"] = [0]
    else: 
         kw_result_df["Verb_VariationI"] = [round(len(list(set(verb_ws)))/len(verb_ws), 3)]
    
    if len(verb_ws) == 0 or len(content_words) == 0: 
        kw_result_df["Verb_VariationII"] = [0]
    else: 
        kw_result_df["Verb_VariationII"] = [round(len(list(set(verb_ws)))/len(content_words), 3)]
    
    cc_ts = opencc.OpenCC('tw2s')
    
    words = [cc_ts.convert(w) for w in intro_segs_uni]
    result = emotion.emotion_count(" ".join(words))
    
    kw_result_df["Preference"] = [round(result.get("好")/len(words), 3)]
    kw_result_df["Happiness"] = [round(result.get("乐")/len(words), 3)]
    kw_result_df["Sadness"] = [round(result.get("哀")/len(words), 3)]
    kw_result_df["Anger"] = [round(result.get("怒")/len(words), 3)]
    kw_result_df["Afraid"] = [round(result.get("惧")/len(words), 3)]
    kw_result_df["Hate"] = [round(result.get("恶")/len(words), 3)]
    kw_result_df["Suprise"] = [round(result.get("惊")/len(words), 3)]

    normal_kws = []
    bs_kws = []
    tea_kws = []
    normal_kws_sims = []
    bs_kws_sims = []
    tea_kws_sims = []

    for t in kws:
        if re.search("_", t):
            if len(t.split("_")) == 2:
                words = "".join(intro_segs_bi).split("/")
                similarities = []
                sim_ws = []
                for w in words:
                    try:
                        sim = w2v_model.wv.similarity(t, w)
                        similarities.append(sim)
                        sim_ws.append(w)
                    except:
                        pass
                if len(similarities) > 0:
                    kw_result_df[t] = [max(similarities)]
                    k_index = similarities.index(max(similarities))
                else:
                    kw_result_df[t] = [0]
            elif len(t.split("_")) == 3:
                words = "".join(intro_segs_tri).split("/")
                similarities = []
                sim_ws = []
                for w in words:
                    try:
                        sim = w2v_model.wv.similarity(t, w)
                        similarities.append(sim)
                        sim_ws.append(w)
                    except:
                        pass
                if len(similarities) > 0: 
                    kw_result_df[t] = [max(similarities)]
                    k_index = similarities.index(max(similarities))
                else:
                    kw_result_df[t] = [0]
        else:
            words = "".join(intro_segs_uni).split("/")
            similarities = []
            sim_ws = []
            for w in words:
                try:
                    sim = w2v_model.wv.similarity(t, w)
                    similarities.append(sim)
                    sim_ws.append(w)
                except:
                    pass
            if len(similarities) > 0: 
                kw_result_df[t] = [max(similarities)]
                k_index = similarities.index(max(similarities))
            else:
                kw_result_df[t] = [0]

    features = model.feature_names
    kw_result_df = kw_result_df.reindex(columns=features)
    preds = np.zeros((kw_result_df.shape[0], 3))
    preds = model.predict(xgb.DMatrix(kw_result_df))
    
    ptf_percentage = []
    for pft in preds:
        p_percentage = []
        for p in pft:
            p_percentage.append(str((Decimal(str(p))*Decimal(str(100))).quantize(Decimal('.00'), ROUND_HALF_UP))+"%")
        ptf_percentage.append(p_percentage)

    out_df = pd.DataFrame(ptf_percentage)
    out_df.columns = ['一般', '熱銷', '歷久彌新']
    return out_df


C:\Users\hhfeng\anaconda3\envs\BS-v2\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import google.generativeai as genai
import os

api_key = "YOUR_GEMINI_API_KEY"
genai.configure(api_key = api_key)

gemini = genai.GenerativeModel('gemini-pro')


In [6]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from openai import OpenAI
OPENAI_API_KEY = 'YOUR_OPENAI_API_KEY'
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
client = OpenAI()
llm = Ollama(model="znbang/breeze:7b-instruct-v0.1-Q8_0")
copy = {}
book = open("book.txt", encoding="UTF-8").read()
query = f"""你現在是一個專業的文案寫作專家，請寫下你經由思考過後，精心設計的文案。請根據使用者的指令跟需求給予相應的幫助。請根據文步的定義及內容，撰寫高品質，有吸引力的該文步內容。無需給予其他不相關的說明跟敘述。文本書籍：{book}"""

moves_def = ["行銷語","內容摘要","本書特色","名人推薦","特別收錄"]
experts = {"行銷語":{}, "內容摘要":{}, "本書特色":{},"名人推薦":{}, "特別收錄":{}}

def llm_eval(a,b,c):
    completion = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system",
         "content": "你是一個只會說繁體中文的助手，請根據使用者的指令跟需求給予相應的幫助。請根據文步的定義及內容，撰寫高品質，有吸引力的該文步內容。"},
        {"role": "user", "content":f"""下列有三個專家的書本行銷文案，請幫我各自依照不同的標準給它分數，以阿拉伯數字1-10分，不用給我其他無關的敘述跟說明。
        參考標準如下：[1.清晰性和相關性:, 2.吸引力和說服力, 3.行動呼籲]。
        清晰性與相關性定義: 指書籍銷售文案如何清楚且相關地傳達書籍的本質和吸引力。清晰和相關的銷售文案可以幫助潛在讀者迅速且精確地理解書籍的內容及其重要性。
        吸引力與說服力定義: 此方面專注於書籍銷售文案的吸引力和說服力。它涉及吸引讀者的興趣並說服他們書籍的價值，從而影響他們的購買決定。
        行動呼籲定義: 書籍銷售文案中的行動呼籲是直接請求或建議，鼓勵讀者立即採取行動，例如購買書籍或進一步了解。有效的行動呼籲對於將興趣轉化為實際銷售至關重要。
        生成格式如下:
        [專家A:清晰性和相關性:分數,參與度和說服力:分數,行動呼籲:分數, 文步文本],
        [專家B:清晰性和相關性:分數,參與度和說服力:分數,行動呼籲:分數, 文步文本], 
        [專家C:清晰性和相關性:分數,參與度和說服力:分數,行動呼籲:分數, 文步文本]。{a},{b},{c}"""}
      ]
    )
    return completion.choices[0].message.content

def decode_unicode(unicode_string):
    # Decode the unicode string to readable characters
    decoded_string = unicode_string.encode().decode('unicode_escape')
    return decoded_string
    
def generate_text(prompt):
    res = gemini.generate_content(prompt)

    return res

def best_BS_decision(move):
    if move == "行銷語":
        res = m1_res
    elif move == "內容摘要":
        res = m2_res
    elif move == "本書特色":
        res = m3_res
    elif move == "名人推薦":
        res = m4_res
    elif move == "特別收錄":
        res = m5_res

    expert_A_avg = re.findall("專家A:.*(\d+).*(\d+).*(\d+)",res)[0]
    expert_B_avg = re.findall("專家B:.*(\d+).*(\d+).*(\d+)",res)[0]    
    expert_C_avg = re.findall("專家C:.*(\d+).*(\d+).*(\d+)",res)[0]    

    expert_A_score = round(sum(int(num) for num in expert_A_avg)/3,2)
    expert_B_score = round(sum(int(num) for num in expert_B_avg)/3,2)
    expert_C_score = round(sum(int(num) for num in expert_C_avg)/3,2)
    
    expert_A_intro = decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts[move]["專家A"]))[0])
    expert_B_intro = decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts[move]["專家B"]))[0])
    expert_C_intro = decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts[move]["專家C"]))[0])
    
    expert_A_BSpred = BS_prediction(expert_A_intro)
    expert_B_BSpred = BS_prediction(expert_B_intro)
    expert_C_BSpred = BS_prediction(expert_C_intro)
    
    A_pred_score = round(float(expert_A_BSpred["歷久彌新"][0].strip("%"))/10*1+float(expert_A_BSpred["熱銷"][0].strip("%"))/10*0.5, 2)
    B_pred_score = round(float(expert_B_BSpred["歷久彌新"][0].strip("%"))/10*1+float(expert_B_BSpred["熱銷"][0].strip("%"))/10*0.5, 2)
    C_pred_score = round(float(expert_C_BSpred["歷久彌新"][0].strip("%"))/10*1+float(expert_C_BSpred["熱銷"][0].strip("%"))/10*0.5, 2)
    
    A_final_score = expert_A_score+A_pred_score
    B_final_score = expert_B_score+B_pred_score
    C_final_score = expert_C_score+C_pred_score
    
    score_list = [A_final_score, B_final_score, C_final_score]
    best_id = score_list.index(max(score_list))
    
    if best_id == 0:
        return expert_A_intro
    elif best_id == 1:
        return expert_B_intro
    elif best_id == 2:
        return expert_C_intro
        

def BS_score(move):
    if move == "行銷語":
        res = m1_res
    elif move == "內容摘要":
        res = m2_res
    elif move == "本書特色":
        res = m3_res
    elif move == "名人推薦":
        res = m4_res
    elif move == "特別收錄":
        res = m5_res

    expert_A_avg = re.findall("專家A:.*(\d+).*(\d+).*(\d+)",res)[0]
    expert_B_avg = re.findall("專家B:.*(\d+).*(\d+).*(\d+)",res)[0]    
    expert_C_avg = re.findall("專家C:.*(\d+).*(\d+).*(\d+)",res)[0]    

    expert_A_score = round(sum(int(num) for num in expert_A_avg)/3,2)
    expert_B_score = round(sum(int(num) for num in expert_B_avg)/3,2)
    expert_C_score = round(sum(int(num) for num in expert_C_avg)/3,2)
    
    expert_A_intro = decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts[move]["專家A"]))[0])
    expert_B_intro = decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts[move]["專家B"]))[0])
    expert_C_intro = decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts[move]["專家C"]))[0])
    
    expert_A_BSpred = BS_prediction(expert_A_intro)
    expert_B_BSpred = BS_prediction(expert_B_intro)
    expert_C_BSpred = BS_prediction(expert_C_intro)
    
    A_pred_score = round(float(expert_A_BSpred["歷久彌新"][0].strip("%"))/10*1+float(expert_A_BSpred["熱銷"][0].strip("%"))/10*0.5, 2)
    B_pred_score = round(float(expert_B_BSpred["歷久彌新"][0].strip("%"))/10*1+float(expert_B_BSpred["熱銷"][0].strip("%"))/10*0.5, 2)
    C_pred_score = round(float(expert_C_BSpred["歷久彌新"][0].strip("%"))/10*1+float(expert_C_BSpred["熱銷"][0].strip("%"))/10*0.5, 2)
    
    A_final_score = expert_A_score+A_pred_score
    B_final_score = expert_B_score+B_pred_score
    C_final_score = expert_C_score+C_pred_score
    
    
    return A_final_score, B_final_score, C_final_score

for move in range(5):
    if move == 0:
        prompt= f"{query}, 請幫我生成 {moves_def[0]}的文步內容，行銷語目的在於吸引讀者注意及引發興趣，並使讀者在閱讀接下來的文案之前，可以快速進入該書之語境。此文步位於開頭，常搭配符號，如「★」。"

        expert_a = generate_text(prompt)
        expert_b = generate_text(prompt)
        expert_c = generate_text(prompt)

        
        experts["行銷語"]["專家A"] = expert_a
        experts["行銷語"]["專家B"] = expert_b
        experts["行銷語"]["專家C"] = expert_c

        ###after-eval
        m1_res = llm_eval(expert_a, expert_b, expert_c)
        m1_final = best_BS_decision(moves_def[0])
        copy["行銷語"] = m1_final

    elif move == 1:
        prompt = f"{query}, 目前已生成好的文本如下：行銷術語：{copy['行銷語']}，請根據目前已生成好的文步內容，繼續思考下一個文步內容的撰寫。下一個欲生成的文步： {moves_def[1]} 。內容摘要旨在介紹整本書之重點內容摘要或背景知識，以讓讀者了解整本書所圍繞之重點。"
        expert_a = generate_text(prompt)
        expert_b = generate_text(prompt)
        expert_c = generate_text(prompt)
        
        experts["內容摘要"]["專家A"] = expert_a
        experts["內容摘要"]["專家B"] = expert_b
        experts["內容摘要"]["專家C"] = expert_c

        ###after-eval

        m2_res = llm_eval(expert_a, expert_b, expert_c)

        m2_final = best_BS_decision(moves_def[1])
        copy["內容摘要"] = m2_final
        
    elif move == 2:
        prompt = f"{query}, 目前已生成好的文本如下：行銷語：{copy['行銷語']}，內容摘要：{copy['內容摘要']}，請再根據目前已生成好的文步繼續思考下一個文步內容的撰寫。下一個欲生成的文步： {moves_def[2]}。本書特色是用於介紹該書與其他書的差異，以讓讀者了解到，藉由閱讀該書後，可以學到從類似書籍所不能習得的新知識。"
        
        experts["本書特色"]["專家A"] = generate_text(prompt)
        experts["本書特色"]["專家B"] = generate_text(prompt)
        experts["本書特色"]["專家C"] = generate_text(prompt)

        ###after-eval

        m3_res = llm_eval(expert_a, expert_b, expert_c)

        m3_final = best_BS_decision(moves_def[2])
        copy["本書特色"] = m3_final
        
    elif move == 3:
        prompt = f"{query}, 目前已生成好的文本如下：行銷語：{copy['行銷語']}，內容摘要：{copy['內容摘要']}，本書特色：{copy['本書特色']}，請再根據目前已生成好的文步繼續思考下一個文步內容的撰寫。下一個欲生成的文步： {moves_def[3]}。名人推薦在於介紹該書得到哪些名人的推薦，包含推薦語。"
        
        experts["名人推薦"]["專家A"] = generate_text(prompt)
        experts["名人推薦"]["專家B"] = generate_text(prompt)
        experts["名人推薦"]["專家C"] = generate_text(prompt)
        
        ###after-eval
        m4_res = llm_eval(expert_a, expert_b, expert_c)
        m4_final = best_BS_decision(moves_def[3])
        copy["名人推薦"] = m4_final
    
    elif move == 4:
        prompt = f"{query}, 目前已生成好的文本如下：行銷語：{copy['行銷語']}，內容摘要：{copy['內容摘要']}，本書特色：{copy['本書特色']}，{copy['名人推薦']}，請再根據目前已生成好的文步繼續思考下一個文步內容的撰寫。下一個欲生成的文步： {moves_def[4]}。特別收錄在介紹特別隨書附贈之圖表。請統一以String形式生成。"
        
        experts["特別收錄"]["專家A"] = generate_text(prompt)
        experts["特別收錄"]["專家B"] = generate_text(prompt)
        experts["特別收錄"]["專家C"] = generate_text(prompt)
        
        ###after-eval
        m5_res = llm_eval(expert_a, expert_b, expert_c)

        m5_final = best_BS_decision(moves_def[4])

        copy["特別收錄"] = m5_final

In [7]:
print(copy["行銷語"])
print(copy["內容摘要"])
print(copy["本書特色"])
print(copy["名人推薦"])
print(copy["特別收錄"])

★ 解開財富密碼：成為理財高手，打造財務自由之路！
**內容摘要**

踏入財務管理的世界，揭開理財致富的奧秘！

本書涵蓋一系列財經知識，從個人理財的基礎，到投資策略的進階應用，為您提供全方位的財務指南。您將學習如何：

* 掌握預算和儲蓄，為您的財務目標奠定基礎
* 有效管理債務，避免陷入財務困境
* 利用保險保障您的財務未來
* 識別和利用稅務優惠，增加您的投資回報

深入投資領域，本書將探討股票、債券、共同基金和房地產投資的方方面面，幫助您建立多元化的投資組合。此外，您還將了解風險管理的重要性，以及如何使用對沖策略和技術分析來提高您的投資績效。

本書旨在為您提供實用的財務知識，讓您具備必要的技能，掌控您的財務，實現財務自由和財富增值。無論您是財務新手還是經驗豐富的投資者，這本書都能讓您受益匪淺，踏上財務成功之路。
**本書特色**

與眾不同的財經指南，助您脫穎而出：

* **全面實用：**從基礎財務管理到進階投資策略，涵蓋所有重要的財經知識。
* **深入淺出：**以清晰易懂的語言闡述複雜的概念，讓初學者也能輕鬆入門。
* **案例豐富：**結合實際案例，幫助您將理論知識應用於現實生活中。
* **專業觀點：**彙集業界專家的見解，提供寶貴的投資策略和財務建議。
* **全方位指導：**從個人理財到全球經濟，為您的財務規劃提供全方位的指導。

選擇本書，您不僅能獲得財經知識，更能獲得理財致富的實用技能，助您踏上財務自由之路，創造財富豐盛的人生！
**名人推薦**

「這本書是財務管理的寶典，為個人理財和投資提供全方位的指導。它將幫助您掌控您的財務，實現財務自由和財富增值。」

——理財暢銷書作者 約翰·博格

「對於任何想要提高財務素養和投資知識的人來說，這本書都是必不可少的讀物。它以清晰易懂的方式涵蓋了所有重要的財經概念。」

——《華爾街日報》專欄作家 沃倫·巴菲特

「我強烈推薦這本書給所有想要掌控自己的財務未來的人。它充滿了實用的建議和有價值的見解，將幫助您實現您的財務目標。」

——投資大師 彼得·林奇
**特別收錄**

* 隨書附贈：財務規劃工作表和投資組合追蹤器，幫助您制定理財計畫和管理投資。
* 專屬線上資源：提供額外的文章、影片和互動工具，讓您深入探索財經世界。


In [37]:
final = copy["內容摘要"] + copy["內容摘要"] + copy["本書特色"] + copy["名人推薦"] + copy["特別收錄"]

**內容摘要**

踏入財務管理的世界，揭開理財致富的奧秘！

本書涵蓋一系列財經知識，從個人理財的基礎，到投資策略的進階應用，為您提供全方位的財務指南。您將學習如何：

* 掌握預算和儲蓄，為您的財務目標奠定基礎
* 有效管理債務，避免陷入財務困境
* 利用保險保障您的財務未來
* 識別和利用稅務優惠，增加您的投資回報

深入投資領域，本書將探討股票、債券、共同基金和房地產投資的方方面面，幫助您建立多元化的投資組合。此外，您還將了解風險管理的重要性，以及如何使用對沖策略和技術分析來提高您的投資績效。

本書旨在為您提供實用的財務知識，讓您具備必要的技能，掌控您的財務，實現財務自由和財富增值。無論您是財務新手還是經驗豐富的投資者，這本書都能讓您受益匪淺，踏上財務成功之路。**內容摘要**

踏入財務管理的世界，揭開理財致富的奧秘！

本書涵蓋一系列財經知識，從個人理財的基礎，到投資策略的進階應用，為您提供全方位的財務指南。您將學習如何：

* 掌握預算和儲蓄，為您的財務目標奠定基礎
* 有效管理債務，避免陷入財務困境
* 利用保險保障您的財務未來
* 識別和利用稅務優惠，增加您的投資回報

深入投資領域，本書將探討股票、債券、共同基金和房地產投資的方方面面，幫助您建立多元化的投資組合。此外，您還將了解風險管理的重要性，以及如何使用對沖策略和技術分析來提高您的投資績效。

本書旨在為您提供實用的財務知識，讓您具備必要的技能，掌控您的財務，實現財務自由和財富增值。無論您是財務新手還是經驗豐富的投資者，這本書都能讓您受益匪淺，踏上財務成功之路。**本書特色**

與眾不同的財經指南，助您脫穎而出：

* **全面實用：**從基礎財務管理到進階投資策略，涵蓋所有重要的財經知識。
* **深入淺出：**以清晰易懂的語言闡述複雜的概念，讓初學者也能輕鬆入門。
* **案例豐富：**結合實際案例，幫助您將理論知識應用於現實生活中。
* **專業觀點：**彙集業界專家的見解，提供寶貴的投資策略和財務建議。
* **全方位指導：**從個人理財到全球經濟，為您的財務規劃提供全方位的指導。

選擇本書，您不僅能獲得財經知識，更能獲得理財致富的實用技能，助您踏上財務自由之路，創造財富豐盛的人生！**名人推薦**

「這本書是財務管理的寶典，為個人理財和投資提供全方位的指導。它將幫助您掌控您

In [38]:
import pandas as pd 
book_copies = pd.DataFrame({"行銷語":{"專家A":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["行銷語"]["專家A"]))[0]),
                                      "專家B":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["行銷語"]["專家B"]))[0]),
                                      "專家C":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["行銷語"]["專家C"]))[0])},
                           "內容摘要":{"專家A":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["內容摘要"]["專家A"]))[0]),
                                      "專家B":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["內容摘要"]["專家B"]))[0]),
                                      "專家C":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["內容摘要"]["專家C"]))[0])},
                           "本書特色":{"專家A":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["本書特色"]["專家A"]))[0]),
                                       "專家B":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["本書特色"]["專家B"]))[0]),
                                       "專家C":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["本書特色"]["專家C"]))[0])},
                           "名人推薦":{"專家A":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["名人推薦"]["專家A"]))[0]),
                                       "專家B":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["名人推薦"]["專家B"]))[0]),
                                       "專家C":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["名人推薦"]["專家C"]))[0])},
                           "特別收錄":{"專家A":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["特別收錄"]["專家A"]))[0]),
                                       "專家B":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["特別收錄"]["專家B"]))[0]),
                                       "專家C":decode_unicode(re.findall("text\"\:\s\"(.*)\"", str(experts["特別收錄"]["專家C"]))[0])},
                           "行銷語分數":{"專家A":BS_score("行銷語")[0],
                                       "專家B":BS_score("行銷語")[1],
                                       "專家C":BS_score("行銷語")[2]},
                           "內容摘要分數":{"專家A":BS_score("內容摘要")[0],
                                       "專家B":BS_score("內容摘要")[1],
                                       "專家C":BS_score("內容摘要")[2]},
                           "本書特色分數":{"專家A":BS_score("本書特色")[0],
                                       "專家B":BS_score("本書特色")[1],
                                       "專家C":BS_score("本書特色")[2]},
                           "名人推薦分數":{"專家A":BS_score("名人推薦")[0],
                                       "專家B":BS_score("名人推薦")[1],
                                       "專家C":BS_score("名人推薦")[2]},
                           "特別收錄分數":{"專家A":BS_score("特別收錄")[0],
                                       "專家B":BS_score("特別收錄")[1],
                                       "專家C":BS_score("特別收錄")[2]},
                           "最終文案":final})

In [41]:
book_copies.to_excel("book_copy.xlsx")